In [2]:
# Import section
import cv2
import numpy as np
from os import scandir, getcwd, rename
import requests
import time
import json
import re
import pandas as pd

In [22]:
def get_cnn_value_from_json(json_local):
    json.load(json_local)

def stringfy(classes):
    r = ""
    for i in classes:
        r += str(i)
    return r.strip()
def aciertos_from_str(real, est):
    count = 0
    for i in range(len(real)):
        if real[i]==est[i]:
            count += 1
    return count

def distance(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

def improve(num, num_pred, scores):
    res = ""
    for i in range(len(str(num))):
        if scores[i] >= 9400:
            res += str(str(num)[i])
        else:
            res += str(str(num_pred)[i])
    return res

def has_letters(word):
    ret = True
    for c in word:
        if not ('0' <= c and c <= '9'):
            return not False
    return not True

def clean_letters(word):
    ret = ""
    for c in word:
        if ('0' <= c and c <= '9'):
            ret += str(c)
    return ret

def clean_real_dni(word):
    word = word.replace("-","")
    word = word.replace("O","0")
    word = word.replace("o","0")
    word = word.replace(" ","")
    return word

np_aciertos_from_str = np.vectorize(aciertos_from_str)
np_distance = np.vectorize(distance)
np_improve = np.vectorize(improve)
np_has_letters = np.vectorize(has_letters)

# REAL

In [4]:
m3100_real = pd.read_csv('D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\process\\muestra900\\levantamiento\\total.csv', dtype={'DNI':str, 'telefono':str})
m3100_real = m3100_real.fillna('')
m3100_real['NombreArchivo'] = m3100_real['nombre_archivo'].apply(lambda x: x[:-4])
m3100_real.head()

,Unnamed: 0,nombre_archivo,DNI,telefono,Observación,email,NombreArchivo
0,0,MarzoCompras_0202072019164448_00043.jpg,42650934,979754672,,Maicusi@hotmail.com,MarzoCompras_0202072019164448_00043
1,1,MarzoCompras_0202072019164454_00245.jpg,20996780,923540008,,,MarzoCompras_0202072019164454_00245
2,2,MarzoCompras_0202072019164458_00359.jpg,42051024,940445230,,jtrujillowong@outlook.com,MarzoCompras_0202072019164458_00359
3,3,MarzoCompras_0202072019164503_00537.jpg,40557762,987136744,,Micasu -80@Hotmail.com,MarzoCompras_0202072019164503_00537
4,4,MarzoCompras_0202072019164503_00546.jpg,08668202,995382267,,CAROLINA CM10_10@HOTMAIL.COM,MarzoCompras_0202072019164503_00546


# ESTIMADO

In [74]:
bd_azure = pd.read_csv('azure_result/azure_result_metrics_afterclean.csv', dtype={'DNI':str, 'telefono':str})
bd_azure = bd_azure.fillna('')

In [131]:
total_comp_azure = pd.merge(bd_azure, m3100_real, how='left', on='NombreArchivo')
total_comp_azure['dni_lev_azure'] = np_distance(total_comp_azure['DNI_x'],total_comp_azure['DNI_y'])
total_comp_azure['tel_lev_azure'] = np_distance(total_comp_azure['Telefono'],total_comp_azure['telefono'])

total_comp_azure['email'] = np.where(total_comp_azure['email']=="-",
                                    "",
                                    total_comp_azure['email'].apply(str.lower))
total_comp_azure['Correo'] = total_comp_azure['Correo'].apply(str.lower)
total_comp_azure['mail_lev_azure'] = np_distance(total_comp_azure['Correo'],total_comp_azure['email'])

# Cleaning muestra
total_comp_azure['DNI_y'] = total_comp_azure['DNI_y'].apply(str.strip)
total_comp_azure['DNI_y'] = total_comp_azure['DNI_y'].apply(clean_real_dni)

# Columnas con letras
total_comp_azure['letter_in_DNI_real'] = np_has_letters(total_comp_azure['DNI_y'])
total_comp_azure['letter_in_DNI_est'] = np_has_letters(total_comp_azure['DNI_x'])

# Longitud de estimación
total_comp_azure['len_DNI_est'] = total_comp_azure['DNI_x'].apply(len)
total_comp_azure['len_tel_est'] = total_comp_azure['Telefono'].apply(len)

# New rule test
total_comp_azure['DNI_final'] = np.where(total_comp_azure.len_DNI_est>=14,
                                       total_comp_azure['DNI_x'].apply(lambda x : clean_letters(x)[:8]),
                                       total_comp_azure['DNI_x']
                                      )

# Longitud de estimación
total_comp_azure['len_DNI_est_fin'] = total_comp_azure['DNI_final'].apply(len)

total_comp_azure['dni_lev_azure_fin'] = np_distance(total_comp_azure['DNI_final'],total_comp_azure['DNI_y'])

In [132]:
total_comp_azure.columns

Index(['NombreArchivo', 'DNI_x', 'AcertividadDNI', 'Telefono',
       'AcertividadTelefono', 'NombreCompleto', 'AcertividadNombreCompleto',
       'Direccion', 'AcertividadDireccion', 'Distrito', 'AcertividadDistrito',
       'Correo', 'AcertividadCorreo', 'Unnamed: 0', 'nombre_archivo', 'DNI_y',
       'telefono', 'Observación', 'email', 'dni_lev_azure', 'tel_lev_azure',
       'mail_lev_azure', 'letter_in_DNI_real', 'letter_in_DNI_est',
       'len_DNI_est', 'len_tel_est', 'DNI_final', 'len_DNI_est_fin',
       'dni_lev_azure_fin'],
      dtype='object')

## Detalle DNI

In [100]:
total_comp_azure[['NombreArchivo','DNI_y','DNI_final','dni_lev_azure_fin']][total_comp_azure.len_DNI_est==1]

,NombreArchivo,DNI_y,DNI_final,dni_lev_azure_fin
0,MarzoCompras-0303072019121007_00008,000672966,100672966,1
1,MarzoCompras-0303072019121007_00010,09888016,09883016,1
4,MarzoCompras-0303072019121014_00331,08281360,08281361,1
7,MarzoCompras-0303072019121017_00459,08482279,08982279,1
8,MarzoCompras-0303072019121018_00497,45279490,45277490,1
21,MarzoCompras-0303072019121032_00998,161145220,16145220,1
22,MarzoCompras-0303072019125009_00002,71112750,71412750,1
29,MarzoCompras-0303072019125015_00264,06163611,:06163611,1
33,MarzoCompras-0303072019125023_00541,48388382,48388282,1
34,MarzoCompras-0303072019125027_00655,147395326,14739532,1


In [50]:
lev_dni_azure = total_comp_azure.groupby(['len_DNI_est']).agg({'NombreArchivo':'count'})
lev_dni_azure['porcentaje'] = np.round(lev_dni_azure['NombreArchivo'] * 100 / lev_dni_azure['NombreArchivo'].sum() , 2)
lev_dni_azure['cuenta'] = lev_dni_azure['NombreArchivo']
lev_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
len_DNI_est,,
0,88,2.78
1,25,0.79
2,2,0.06
3,2,0.06
4,1,0.03
5,1,0.03
6,9,0.28
7,170,5.38
8,2389,75.60


## Detalle Telefono 

In [127]:
total_comp_azure[['NombreArchivo','Telefono','telefono','tel_lev_azure']][total_comp_azure.len_tel_est==0]

,NombreArchivo,Telefono,telefono,tel_lev_azure
122,MarzoCompras_0202072019164511_00837,98083440,980834403,1
153,MarzoCompras_0203072019122719_00317,99803167,99903167,1
166,MarzoCompras_0203072019122734_00881,2764230,2764230,0
182,MarzoCompras_0203072019130749_00573,8975217JJ,997521755,3
191,MarzoCompras_AV_B3_20190705171107_00218,,5311830,7
241,MarzoCompras_AV_B3_20190708093427_00570,,991589375,9
245,MarzoCompras_AV_B3_20190708093431_00718,,989568657,9
264,MarzoCompras_AV_B3_20190708100700_00554,DN:19825169964646696,-,20
283,MarzoCompras_AV_B3_20190708103535_00284,906207081,986207081,1
285,MarzoCompras_AV_B3_20190708103538_00427,,,0


## Detalle E-mail 

In [133]:
total_comp_azure[['NombreArchivo','Correo','email','mail_lev_azure']][total_comp_azure.mail_lev_azure==1]

,NombreArchivo,Correo,email,mail_lev_azure
41,MarzoCompras-0303072019132555_00124,pattychic_14.7@hotmail.com,pattychic_14_7@hotmail.com,1
45,MarzoCompras-0303072019132602_00385,alegna-19192@hotmail.com,alegna_19192@hotmail.com,1
61,MarzoCompras-0303072019132618_01004,renzorojasperu456@gmailcom,renzorojasperu456@gmail.com,1
132,MarzoCompras_0202072019182422_00336,dario-2203_19@hotmail.com,dario_2203_19@hotmail.com,1
172,MarzoCompras_0203072019130737_00134,diazmarin93@gmail.con,diazmarin93@gmail.com,1
200,MarzoCompras_AV_B3_20190705171124_00762,rojasaguirre@hotmail.com,rrojasaguirre@hotmail.com,1
218,MarzoCompras_AV_B3_20190705174256_00916,camucha-lora@hotmail.com,camucha_lora@hotmail.com,1
219,MarzoCompras_AV_B3_20190705174257_00961,daniebolanos@gmail.com,daniebolanosc@gmail.com,1
247,MarzoCompras_AV_B3_20190708093435_00850,norma23-12@hotmail.com,norma23:12@hotmail.com,1
263,MarzoCompras_AV_B3_20190708100659_00522,rosa.marquez.imf@gmail.com,rosa.marquez.rmf@gmail.com,1


## Medicion DNI

In [78]:
lev_dni_azure = total_comp_azure.groupby(['dni_lev_azure_fin']).agg({'NombreArchivo':'count'})
lev_dni_azure['porcentaje'] = np.round(lev_dni_azure['NombreArchivo'] * 100 / lev_dni_azure['NombreArchivo'].sum() , 2)
lev_dni_azure['cuenta'] = lev_dni_azure['NombreArchivo']
lev_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
dni_lev_azure_fin,,
0,1847,58.45
1,728,23.04
2,280,8.86
3,104,3.29
4,46,1.46
5,20,0.63
6,15,0.47
7,20,0.63
8,92,2.91


## Medición Telefono

In [79]:
tel_dni_azure = total_comp_azure.groupby(['tel_lev_azure']).agg({'NombreArchivo':'count'})
tel_dni_azure['porcentaje'] = np.round(tel_dni_azure['NombreArchivo'] * 100 / tel_dni_azure['NombreArchivo'].sum() , 2)
tel_dni_azure['cuenta'] = tel_dni_azure['NombreArchivo']
tel_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
tel_lev_azure,,
0,1705,53.96
1,833,26.36
2,311,9.84
3,118,3.73
4,51,1.61
5,10,0.32
6,9,0.28
7,19,0.60
8,13,0.41


## Medición E-mail

In [134]:
mail_dni_azure = total_comp_azure.groupby(['mail_lev_azure']).agg({'NombreArchivo':'count'})
mail_dni_azure['porcentaje'] = np.round(mail_dni_azure['NombreArchivo'] * 100 / mail_dni_azure['NombreArchivo'].sum() , 2)
mail_dni_azure['cuenta'] = mail_dni_azure['NombreArchivo']
mail_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
mail_lev_azure,,
0,1325,41.93
1,228,7.22
2,238,7.53
3,214,6.77
4,202,6.39
5,184,5.82
6,153,4.84
7,122,3.86
8,94,2.97


## Parametro DNI

In [81]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','DNI_y','DNI_final','AcertividadDNI']]
    # param_dni = param_dni[param_dni.dni_yolo.apply(len)==8]
    param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadDNI']>=i,param_dni.DNI_final.apply(len)==8), 1, 0)
    param_dni['trues_real'] = np.where(param_dni['DNI_final']==param_dni['DNI_y'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


40 se tiene 0.84 0.58
41 se tiene 0.84 0.58
42 se tiene 0.84 0.58
43 se tiene 0.84 0.58
44 se tiene 0.84 0.58
45 se tiene 0.84 0.58
46 se tiene 0.84 0.58
47 se tiene 0.84 0.58
48 se tiene 0.84 0.58
49 se tiene 0.84 0.58
50 se tiene 0.84 0.58
51 se tiene 0.84 0.58
52 se tiene 0.84 0.58
53 se tiene 0.84 0.58
54 se tiene 0.84 0.58
55 se tiene 0.84 0.58
56 se tiene 0.84 0.58
57 se tiene 0.84 0.58
58 se tiene 0.84 0.58
59 se tiene 0.84 0.58
60 se tiene 0.84 0.58
61 se tiene 0.84 0.58
62 se tiene 0.84 0.58
63 se tiene 0.84 0.58
64 se tiene 0.84 0.58
65 se tiene 0.84 0.58
66 se tiene 0.84 0.58
67 se tiene 0.84 0.58
68 se tiene 0.84 0.58
69 se tiene 0.84 0.58
70 se tiene 0.84 0.58
71 se tiene 0.56 0.58
72 se tiene 0.56 0.58
73 se tiene 0.56 0.58
74 se tiene 0.55 0.58
75 se tiene 0.54 0.58
76 se tiene 0.51 0.58
77 se tiene 0.44 0.58
78 se tiene 0.43 0.58
79 se tiene 0.41 0.58
80 se tiene 0.4 0.58
81 se tiene 0.27 0.58
82 se tiene 0.25 0.58
83 se tiene 0.23 0.58
84 se tiene 0.18 0.58
85 se tiene

In [95]:
param_dni = total_comp_azure[['NombreArchivo','DNI_y','DNI_final','AcertividadDNI']]

param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadDNI']>=67,param_dni.DNI_final.apply(len)==8), 1, 0)
param_dni['trues_real'] = np.where(param_dni['DNI_final']==param_dni['DNI_y'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


trues_real,0,1
trues_est,,
0,463,31
1,850,1816


In [96]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100, 2)

trues_real,0,1
trues_est,,
0,14.65,0.98
1,26.90,57.47


## Parámetro Telefono

In [98]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','Telefono','telefono','AcertividadTelefono','tel_lev_azure']]
    param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadTelefono']>=i,param_dni.Telefono.apply(len)>=6), 1, 0)
    param_dni['trues_real'] = np.where(param_dni['Telefono']==param_dni['telefono'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


40 se tiene 0.97 0.54
41 se tiene 0.97 0.54
42 se tiene 0.97 0.54
43 se tiene 0.97 0.54
44 se tiene 0.97 0.54
45 se tiene 0.97 0.54
46 se tiene 0.97 0.54
47 se tiene 0.97 0.54
48 se tiene 0.97 0.54
49 se tiene 0.97 0.54
50 se tiene 0.97 0.54
51 se tiene 0.97 0.54
52 se tiene 0.97 0.54
53 se tiene 0.97 0.54
54 se tiene 0.97 0.54
55 se tiene 0.97 0.54
56 se tiene 0.97 0.54
57 se tiene 0.97 0.54
58 se tiene 0.97 0.54
59 se tiene 0.97 0.54
60 se tiene 0.97 0.54
61 se tiene 0.97 0.54
62 se tiene 0.97 0.54
63 se tiene 0.97 0.54
64 se tiene 0.97 0.54
65 se tiene 0.97 0.54
66 se tiene 0.97 0.54
67 se tiene 0.97 0.54
68 se tiene 0.97 0.54
69 se tiene 0.97 0.54
70 se tiene 0.97 0.54
71 se tiene 0.65 0.54
72 se tiene 0.65 0.54
73 se tiene 0.64 0.54
74 se tiene 0.63 0.54
75 se tiene 0.61 0.54
76 se tiene 0.58 0.54
77 se tiene 0.51 0.54
78 se tiene 0.5 0.54
79 se tiene 0.47 0.54
80 se tiene 0.46 0.54
81 se tiene 0.3 0.54
82 se tiene 0.29 0.54
83 se tiene 0.27 0.54
84 se tiene 0.19 0.54
85 se tiene 

In [121]:
param_dni['trues_est'] = np.where(np.logical_and(param_dni['AcertividadTelefono']>=77,param_dni.Telefono.apply(len)>=6), 1, 0)
param_dni['trues_real'] = np.where(param_dni['Telefono']==param_dni['telefono'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


trues_real,0,1
trues_est,,
0,903,655
1,552,1050


In [122]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100,2)

trues_real,0,1
trues_est,,
0,28.58,20.73
1,17.47,33.23


## Parámetro E-mail

In [124]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','Correo','email','AcertividadCorreo','mail_lev_azure']]
    param_dni['trues_est'] = np.where(param_dni['AcertividadCorreo']>=i, 1, 0)
    param_dni['trues_real'] = np.where(param_dni['Correo']==param_dni['email'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

40 se tiene 0.6 0.4
41 se tiene 0.6 0.4
42 se tiene 0.6 0.4
43 se tiene 0.6 0.4
44 se tiene 0.6 0.4
45 se tiene 0.6 0.4
46 se tiene 0.6 0.4
47 se tiene 0.6 0.4
48 se tiene 0.6 0.4
49 se tiene 0.6 0.4
50 se tiene 0.6 0.4
51 se tiene 0.6 0.4
52 se tiene 0.6 0.4
53 se tiene 0.6 0.4
54 se tiene 0.6 0.4
55 se tiene 0.6 0.4
56 se tiene 0.6 0.4
57 se tiene 0.6 0.4
58 se tiene 0.6 0.4
59 se tiene 0.6 0.4
60 se tiene 0.6 0.4
61 se tiene 0.6 0.4
62 se tiene 0.6 0.4
63 se tiene 0.6 0.4
64 se tiene 0.6 0.4
65 se tiene 0.6 0.4
66 se tiene 0.6 0.4
67 se tiene 0.6 0.4
68 se tiene 0.6 0.4
69 se tiene 0.6 0.4
70 se tiene 0.6 0.4
71 se tiene 0.4 0.4
72 se tiene 0.4 0.4
73 se tiene 0.39 0.4
74 se tiene 0.37 0.4
75 se tiene 0.32 0.4
76 se tiene 0.25 0.4
77 se tiene 0.18 0.4
78 se tiene 0.18 0.4
79 se tiene 0.14 0.4
80 se tiene 0.14 0.4
81 se tiene 0.08 0.4
82 se tiene 0.08 0.4
83 se tiene 0.06 0.4
84 se tiene 0.04 0.4
85 se tiene 0.04 0.4
86 se tiene 0.03 0.4
87 se tiene 0.03 0.4
88 se tiene 0.02 0.4
89 s

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [125]:
param_dni['trues_est'] = np.where(param_dni['AcertividadCorreo']>=77, 1, 0)
param_dni['trues_real'] = np.where(param_dni['Correo']==param_dni['email'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


trues_real,0,1
trues_est,,
0,1372,1216
1,518,54


In [126]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100,2)

trues_real,0,1
trues_est,,
0,43.42,38.48
1,16.39,1.71
